In [1]:
#Imports 

import numpy as np 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sqlite3
import matplotlib.pyplot as plt
import os

path = "/Users/Aurelien/Desktop/test_jolimoi/data/"
os.chdir(path)

In [2]:
database='artists.db'
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,artists,artists,2,"CREATE TABLE ""artists"" (\n""Artist ID"" INTEGER,..."
1,table,artworks,artworks,159,"CREATE TABLE ""artworks"" (\n""Artwork ID"" INTEGE..."


In [3]:
artists = pd.read_sql("""SELECT *
                FROM artists;""",conn)
artworks =  pd.read_sql("""SELECT *
                FROM artworks;""",conn)

In [4]:
artworks.columns

Index(['Artwork ID', 'Title', 'Artist ID', 'Name', 'Date', 'Medium',
       'Dimensions', 'Acquisition Date', 'Credit', 'Catalogue', 'Department',
       'Classification', 'Object Number', 'Diameter (cm)',
       'Circumference (cm)', 'Height (cm)', 'Length (cm)', 'Width (cm)',
       'Depth (cm)', 'Weight (kg)', 'Duration (s)'],
      dtype='object')

In [5]:
artists.columns

Index(['Artist ID', 'Name', 'Nationality', 'Gender', 'Birth Year',
       'Death Year'],
      dtype='object')

 Which artist in this data set lived the longest?

In [6]:
# the condition on Gender -> there are associations or groups of artists (not human)

pd.read_sql( """ SELECT Name, `Death Year`-`Birth Year` as Age
                 FROM artists
                 WHERE Gender IS NOT NULL AND Age==(SELECT MAX(`Death Year`-`Birth Year`)
                                                      FROM artists
                                                      WHERE (Gender IS NOT NULL));""",conn)

,Name,Age
0,Leni Matthaei,108
1,B. Efimov,108


Who are the top 10 artists by the number of artworks?

In [7]:
# condition on Name -> some values of Name are empty

pd.read_sql( """SELECT Name, COUNT(*) as Nb_artwork
                FROM artworks
                WHERE Name IS NOT NULL
                GROUP BY Name
                ORDER BY Nb_artwork DESC
                LIMIT 10;""",conn)

,Name,Nb_artwork
0,Eugène Atget,5050
1,Louise Bourgeois,3318
2,Ludwig Mies van der Rohe,2566
3,Unknown photographer,1575
4,Jean Dubuffet,1435
5,Lee Friedlander,1317
6,Pablo Picasso,1310
7,Marc Chagall,1162
8,Henri Matisse,1063
9,Pierre Bonnard,894


Which artist is created the most artwork by total surface area?

First idea is to check for 2D art (with Height&Width or Diameter)

In [8]:
pd.read_sql( """SELECT DISTINCT Classification
                FROM artworks
                WHERE (`Height (cm)` IS NOT NULL AND `Width (cm)` IS NOT NULL);""",conn)

,Classification
0,Architecture
1,Mies van der Rohe Archive
2,Design
3,Illustrated Book
4,Print
5,Drawing
6,Film
7,Multiple
8,Periodical
9,Photograph


In [9]:
pd.read_sql( """ SELECT DISTINCT Classification
                FROM artworks
                WHERE (`Diameter (cm)` IS NOT NULL);""",conn)

,Classification
0,Design
1,Illustrated Book
2,Drawing
3,Photograph
4,Print
5,Painting
6,Sculpture
7,Installation
8,Architecture
9,Multiple


In [10]:
# I only kept the following classes : drawing, print, painting, photograph, collage
# I make a union between the table of round objects and the table of rectangular objects
# Then by grouping by Name, we sum all the surfaces created by Name
# And I take the max

pd.read_sql("""Select Name, MAX(`surface (cm2)`) as `max surface (cm2)`
               FROM(
               Select Name, surf as `surface (cm2)`
               
               FROM(
               
               SELECT Name,`Diameter (cm)`*`Diameter (cm)`*3.14 as surf
                FROM artworks
                WHERE (`Diameter (cm)` IS NOT NULL) AND (Classification=='Drawing' OR Classification=='Print'
                                                 OR Classification=='Painting'
                                                 OR Classification=='Photograph'
                                                 OR Classification=='Collage')
                UNION
                
                SELECT Name,`Width (cm)`*`Height (cm)` as surf
                FROM artworks
                WHERE (`Artist ID` IS NOT NULL) AND 
                      (`Width (cm)` IS NOT NULL AND `Height (cm)` IS NOT NULL) AND 
                      (Classification=='Drawing' OR Classification=='Print'
                                                 OR Classification=='Painting'
                                                 OR Classification=='Photograph'
                                                 OR Classification=='Collage'))
                                                 
                GROUP BY Name);""",conn)

,Name,max surface (cm2)
0,Reena Spaulings,297290.922817
